## Extracting unique characters from a dataset

In [2]:
file_path = "/mnt/resource/Etc/data/alamgir12/metadata_normalized.csv"
file_path = "/media/arif/Resource/Etc/data/alamgir12/metadata_normalized.csv"

text = ""
with open(file_path) as f:
    for i, line in enumerate(f):
        _, t, nt = line.split("|")
        text += t

        if i < 5:
            print(t)

characters = set(text)

print(f'Total unique characters: {len(characters)}')
print(f'Characters: {"".join(sorted(characters))}')

 স্কুলটির দিকে দৃষ্টি দিন। 
 পিরোজপুর জেলার কাউখালী বন্দরে একটি মাত্র বালিকা বিদ্যালয় আছে, যা এসবি সরকারি উচ্চবালিকা বিদ্যালয় নামে পরিচিত। 
 অনেক ঐতিহ্য বহন করছে এ বিদ্যালয়টি। 
 কিন্তু এখানকার ছাত্রী ও শিক্ষক-শিক্ষিকাদের সম্মুখীন হতে হচ্ছে বহুবিধ সমস্যার। 
 পাশ্বর্বর্তী বিভিন্ন গ্রাম ও দূরদূরান্ত থেকে ভালো শিক্ষার আশায় এখানে পড়তে আসছে শিক্ষার্থীরা। 
Total unique characters: 152
Characters:  !"#%'()*,-./013568:;?@ABCDEFGHIJKLMNPRSTY[]abcdefghiklmnopqrstuwxy °।ঁংঃঅআইঈউঊঋঌএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহ়ািীুূৃেৈোৌ্ৎৗড়ঢ়য়০১২৩৪৫৬৭৮৯‌‍–—―’”−
